Imports

In [615]:
!pip install -r requirements.txt

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621


In [616]:
import pandas as pd

Load datasets

In [617]:
def read_csv(filename):
    df = pd.read_csv(
        filename,
        header=0,                     # Use the first row as the header
        delimiter=';',                # Use semicolon as the main delimiter
        decimal=',',                  # Specify that commas are used as decimals
        quotechar='"',                # Handle quotes around strings
        skipinitialspace=True,        # Skip spaces after delimiters
    )
    df.set_index(df.columns[0], inplace=True)
    return df

airlines = read_csv('airlines.csv')
airports = read_csv('airports.csv')
planes = read_csv('planes.csv')
flights = read_csv('flights.csv')
weather = read_csv('weather.csv')

In [618]:
planes['year'] = planes['year'].astype('Int64')
flights['dep_time'] = flights['dep_time'].astype('Int64')
flights['dep_delay'] = flights['dep_delay'].astype('Int64')
flights['arr_time'] = flights['arr_time'].astype('Int64')
flights['arr_delay'] = flights['arr_delay'].astype('Int64')

In [619]:
def check_for_nulls(df):
    res = {}
    for c in df.columns:
        if df[df[c].isnull()].shape[0] != 0:
            res[c] = df.shape[0] - df[df[c].isnull()].shape[0]
    return res

In [620]:
print("Nullable columns in airlines:", check_for_nulls(airlines))
print("Nullable columns in airports:", check_for_nulls(airports))
print("Nullable columns in planes:", check_for_nulls(planes))
print("Nullable columns in flights:", check_for_nulls(flights))
print("Nullable columns in weather:", check_for_nulls(weather))

Nullable columns in airlines: {}
Nullable columns in airports: {'tz': 1203, 'dst': 1203, 'tzone': 1132}
Nullable columns in planes: {'year': 4751}
Nullable columns in flights: {'dep_time': 424614, 'dep_delay': 424614, 'arr_time': 423899, 'arr_delay': 422818, 'tailnum': 433439, 'air_time': 422818}
Nullable columns in weather: {'temp': 668, 'dewp': 668, 'humid': 668, 'wind_dir': 24984, 'wind_speed': 25171, 'wind_gust': 25171, 'precip': 1593, 'pressure': 572, 'visib': 26180}


Clean Datasets

In [621]:
def clear_airports(df):
    # I checked that tz is null = dst is null
    df = df.dropna(subset=['tz', 'dst'])

    for idx, row in df[df['tzone'].isnull()].iterrows():
        matching_row = df[(df['tz'] == row['tz']) & pd.notnull(df['tzone'])]
        if not matching_row.empty:
            df.at[idx, 'tzone'] = matching_row['tzone'].iloc[0]
        else:
            df = df.drop(idx)
    return df

In [622]:
def clear_planes(df):
    return df.dropna()

In [623]:
def clear_flights(df):
    # I checked that dep_time is null = dep_delay is null \subset arr_time is null \subset arr_delay is null \subset air_time is null
    df = df.dropna(subset=['tailnum'])

    for idx, row in df.iterrows(): # Fix american format
        df.at[idx, 'dep_time'] = df.at[idx, 'dep_time'] % 100 + (df.at[idx, 'dep_time'] // 100) * 60
        df.at[idx, 'arr_time'] = df.at[idx, 'arr_time'] % 100 + (df.at[idx, 'arr_time'] // 100) * 60
        df.at[idx, 'sched_dep_time'] = df.at[idx, 'sched_dep_time'] % 100 + (df.at[idx, 'sched_dep_time'] // 100) * 60
        df.at[idx, 'sched_arr_time'] = df.at[idx, 'sched_arr_time'] % 100 + (df.at[idx, 'sched_arr_time'] // 100) * 60

    for idx, row in df[df['air_time'].isnull()].iterrows():
        df.at[idx, 'arr_delay'] = df['arr_time'].iloc[0] - df['sched_arr_time'].iloc[0]
        df.at[idx, 'air_time'] = df['arr_time'].iloc[0] - df['dep_time'].iloc[0]
    return df

In [624]:
def clear_weather(df):
    # I checked that precip is null \subset temp is null = dewp is null = humid is null \subset pressure is null
    return df

In [625]:
clean_airlines = airlines
clean_airports = clear_airports(airports.copy())
clean_planes = clear_planes(planes.copy())
clean_flights = clear_flights(flights.copy())
clean_weather = clear_weather(weather.copy())

In [626]:
print("Nullable columns in airlines:", check_for_nulls(clean_airlines))
print("Nullable columns in airports:", check_for_nulls(clean_airports))
print("Nullable columns in planes:", check_for_nulls(clean_planes))
print("Nullable columns in flights:", check_for_nulls(clean_flights))
print("Nullable columns in weather:", check_for_nulls(clean_weather))

Nullable columns in airlines: {}
Nullable columns in airports: {}
Nullable columns in planes: {}
Nullable columns in flights: {'dep_time': 424614, 'dep_delay': 424614, 'arr_time': 423899}
Nullable columns in weather: {'temp': 668, 'dewp': 668, 'humid': 668, 'wind_dir': 24984, 'wind_speed': 25171, 'wind_gust': 25171, 'precip': 1593, 'pressure': 572, 'visib': 26180}


In [627]:
print("Airports old vs new:", airports.shape, clean_airports.shape)
print("Planes old vs new:", planes.shape, clean_planes.shape)
print("Flights old vs new:", flights.shape, clean_flights.shape)
print("Weather old vs new:", weather.shape, clean_weather.shape)

Airports old vs new: (1251, 8) (1194, 8)
Planes old vs new: (4840, 9) (4751, 9)
Flights old vs new: (435352, 19) (433439, 19)
Weather old vs new: (26204, 15) (26204, 15)


Merge datasets

In [628]:
flights_with_origin_info = pd.merge(
    clean_flights,
    clean_airports,
    how='left',
    left_on='origin',
    right_on='faa',
    suffixes=('', '_origin')
)

flights_with_origin_and_dest = pd.merge(
    flights_with_origin_info,
    clean_airports,
    how='inner',
    left_on='dest',
    right_on='faa',
    suffixes=('_origin', '_dest')
)

flights_with_origin_dest_and_planes = pd.merge(
    flights_with_origin_and_dest,
    clean_planes,
    how='left',
    left_on=['tailnum'],
    right_on=['tailnum'],
    suffixes=('', '_plane')
)

merged_df = pd.merge(
    flights_with_origin_dest_and_planes,
    clean_weather,
    how='left',
    left_on=['origin', 'year', 'month', 'day', 'hour'],
    right_on=['origin', 'year', 'month', 'day', 'hour'],
    suffixes=('', '_weather')
)

In [629]:
merged_df = merged_df.drop(['dep_time', 'arr_time', 'flight', 'tailnum', 'air_time', 'time_hour', 'faa_origin', 'name_origin', 'lat_origin', 'lon_origin', 'alt_origin', 'tz_origin', 'dst_origin', 'tzone_origin', 'faa_dest', 'name_dest', 'dst_dest', 'tzone_dest', 'time_hour_weather'], axis=1)


In [630]:
merged_df.to_csv("merged_df.csv", index=False)

In [631]:
ewr_df = merged_df[merged_df['origin'] == 'EWR'].drop('origin', axis=1)
jfk_df = merged_df[merged_df['origin'] == 'JFK'].drop('origin', axis=1)
lga_df = merged_df[merged_df['origin'] == 'LGA'].drop('origin', axis=1)

In [632]:
ewr_delay_df = ewr_df.dropna(subset=['arr_delay'])
ewr_cancel_df = ewr_df
ewr_cancel_df['is_cancelled'] = ewr_cancel_df['dep_delay'].isnull()
ewr_cancel_df = ewr_cancel_df.drop(['dep_delay', 'arr_delay'], axis=1)
jfk_delay_df = jfk_df.dropna(subset=['arr_delay'])
jfk_cancel_df = jfk_df
jfk_cancel_df['is_cancelled'] = jfk_cancel_df['dep_delay'].isnull()
jfk_cancel_df = jfk_cancel_df.drop(['dep_delay', 'arr_delay'], axis=1)
lga_delay_df = lga_df.dropna(subset=['arr_delay'])
lga_cancel_df = lga_df
lga_cancel_df['is_cancelled'] = lga_cancel_df['dep_delay'].isnull()
lga_cancel_df = lga_cancel_df.drop(['dep_delay', 'arr_delay'], axis=1)

In [633]:
ewr_delay_df.to_csv("ewr_delay.csv", index=False)
jfk_delay_df.to_csv("jfk_delay.csv", index=False)
lga_delay_df.to_csv("lga_delay.csv", index=False)
ewr_cancel_df.to_csv("ewr_cancel.csv", index=False)
jfk_cancel_df.to_csv("jfk_cancel.csv", index=False)
lga_cancel_df.to_csv("lga_cancel.csv", index=False)